<a href="https://colab.research.google.com/github/sait1801/grid-test-cases/blob/main/ren_rate_dnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
import pandas as pd
import requests

In [73]:
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [74]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

worksheet = gc.open('consum_3years').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

print(rows)

# Convert to a DataFrame and render.
data = pd.DataFrame.from_records(rows)

[['Date', 'Hour', 'Consumption'], ['01.01.2022', '0:00', '31.021,25'], ['01.01.2022', '1:00', '29.710,69'], ['01.01.2022', '2:00', '28.318,53'], ['01.01.2022', '3:00', '27.169,55'], ['01.01.2022', '4:00', '26.525,16'], ['01.01.2022', '5:00', '26.319,80'], ['01.01.2022', '6:00', '26.809,64'], ['01.01.2022', '7:00', '26.969,94'], ['01.01.2022', '8:00', '27.592,75'], ['01.01.2022', '9:00', '29.365,83'], ['01.01.2022', '10:00', '31.016,83'], ['01.01.2022', '11:00', '32.843,03'], ['01.01.2022', '12:00', '33.557,93'], ['01.01.2022', '13:00', '33.795,67'], ['01.01.2022', '14:00', '33.586,58'], ['01.01.2022', '15:00', '33.220,55'], ['01.01.2022', '16:00', '33.558,85'], ['01.01.2022', '17:00', '35.121,94'], ['01.01.2022', '18:00', '36.687,64'], ['01.01.2022', '19:00', '36.206,53'], ['01.01.2022', '20:00', '35.282,80'], ['01.01.2022', '21:00', '34.354,39'], ['01.01.2022', '22:00', '33.503,32'], ['01.01.2022', '23:00', '31.717,84'], ['02.01.2022', '0:00', '30.615,75'], ['02.01.2022', '1:00', '29.

In [75]:
data.columns = ['Date'	,'Hour'	,'Consumption']
data= data.drop(0)
data.tail(5)

,Date,Hour,Consumption
13868,01.08.2023,19:00,"46.942,86"
13869,01.08.2023,20:00,"48.395,28"
13870,01.08.2023,21:00,"47.683,05"
13871,01.08.2023,22:00,"46.370,95"
13872,01.08.2023,23:00,"43.560,02"


In [76]:
df=data.copy()
df.shape

(13872, 3)

##Adding WeekDay parameter to dataframe. The consumption may change daily due to weekin-weekend realtion

In [77]:
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
df['Weekday'] = 0

In [78]:
weekday_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
for i in range(len(df)):
  row = df.iloc[i]
  weekday = row['Date'].dayofweek
  df.loc[i, 'Weekday'] = weekday


In [79]:
df.tail(5)

,Date,Hour,Consumption,Weekday
13869,2023-08-01,20:00,"48.395,28",1.0
13870,2023-08-01,21:00,"47.683,05",1.0
13871,2023-08-01,22:00,"46.370,95",1.0
13872,2023-08-01,23:00,"43.560,02",0.0
0,NaT,NaN,NaN,5.0


##Changing the format of the energy to MGW

In [80]:
def convert_consumption(consumption):
  """Converts a string consumption to a float."""
  consumption = str(consumption).replace(',', '')
  return float(consumption)

In [81]:
df['Consumption'] = df['Consumption'].apply(convert_consumption)

##Add seasons and weekin-weekday parameters

In [82]:
df['Week_tag'] = 0
for i in range(len(df)):
  row = df.iloc[i]
  weekday = row['Weekday']
  if weekday in ['Sunday', 'Saturday']:
    df.loc[i, 'Week_tag'] = 1 #weekedn
  else:
    df.loc[i, 'Week_tag'] = 0 #weekin



In [83]:
# Assuming your DataFrame is named 'df'
df = df.dropna(subset=['Date'])

# Reset the index after removing the row
df.reset_index(drop=True, inplace=True)
df.tail(5)

,Date,Hour,Consumption,Weekday,Week_tag
13867,2023-08-01,19:00,46.94286,1.0,0
13868,2023-08-01,20:00,48.39528,1.0,0
13869,2023-08-01,21:00,47.68305,1.0,0
13870,2023-08-01,22:00,46.37095,1.0,0
13871,2023-08-01,23:00,43.56002,0.0,0


##ML Training Just by looking these data

In [84]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

df_sample =df.copy()


def convert_hour(hour):
  hour = str(hour)
  """Converts a time string in H:MM format to HH:MM format."""
  if len(hour) == 4:
    hour = '0' + hour[0]
  return hour

df_sample['Hour'] = df_sample['Hour'].apply(convert_hour)
print(df_sample['Hour'])
df_sample['Hour'] = df_sample['Hour'].str[:2].astype('int')

for i in range(1, 6):
    df_sample['previous-{}'.format(i)] = df_sample['Consumption'].shift(i)
for i in range(1, 6):
    df_sample['previous-{}'.format(i)].fillna(df_sample['Consumption'], inplace=True)


# Split the DataFrame into a training set and a test set
train_df = df_sample.sample(frac=0.8, random_state=1)
test_df = df_sample.drop(train_df.index)

# Create the machine learning model
model = LinearRegression()
# print(df_sample['Hour'])
# Train the machine learning model
model.fit(train_df[[ 'Hour', 'Weekday', 'Week_tag','previous-1', 'previous-2','previous-3','previous-4','previous-5']], train_df['Consumption'])

# Predict the Consumption value for the test set
predictions = model.predict(test_df[['Hour', 'Weekday', 'Week_tag','previous-1', 'previous-2','previous-3','previous-4','previous-5']])

##save the model
import pickle

# save the iris classification model as a pickle file
!touch cons_pred.pkl
model_pkl_file = "cons_pred.pkl"

with open(model_pkl_file, 'wb') as file:
    pickle.dump(model, file)

# Evaluate the machine learning model
mse = mean_squared_error(test_df['Consumption'], predictions)
print('MSE:', mse)

0           00
1           01
2           02
3           03
4           04
         ...  
13867    19:00
13868    20:00
13869    21:00
13870    22:00
13871    23:00
Name: Hour, Length: 13872, dtype: object
MSE: 1.1676278258546215


In [85]:
df_sample.dtypes

Date           datetime64[ns]
Hour                    int64
Consumption           float64
Weekday               float64
Week_tag                int64
previous-1            float64
previous-2            float64
previous-3            float64
previous-4            float64
previous-5            float64
dtype: object

##Train DNN

In [86]:
df_sample.head(5)

,Date,Hour,Consumption,Weekday,Week_tag,previous-1,previous-2,previous-3,previous-4,previous-5
0,2022-01-01,0,31.02125,5.0,0,31.02125,31.02125,31.02125,31.02125,31.02125
1,2022-01-01,1,29.71069,5.0,0,31.02125,29.71069,29.71069,29.71069,29.71069
2,2022-01-01,2,28.31853,5.0,0,29.71069,31.02125,28.31853,28.31853,28.31853
3,2022-01-01,3,27.16955,5.0,0,28.31853,29.71069,31.02125,27.16955,27.16955
4,2022-01-01,4,26.52516,5.0,0,27.16955,28.31853,29.71069,31.02125,26.52516


In [87]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import StandardScaler  # Adapt for your scaler


# Preprocess data (adjust based on your specific requirements)
X = df_sample.drop(columns=['Date', 'Consumption'])  # Exclude target variable #The target will be hour or stg ?  'Date', 'Consumption', 'Hour'
scaler_x = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_x.fit_transform(X)

# Reshape data for LSTM input
X_reshaped = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))  # Assuming a single timestep

Y = df_sample[['Consumption']]
Y_scaled = scaler_y.fit_transform(Y)

# Reshape data for LSTM input
Y_reshaped = Y_scaled.reshape((Y_scaled.shape[0], 1, Y_scaled.shape[1]))  # Assuming a single timestep

# Define LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_reshaped.shape[1], X_reshaped.shape[2])))
model.add(LSTM(units=50))
model.add(Dense(1))  # Single unit for regression output


In [88]:
X.head(5)

,Hour,Weekday,Week_tag,previous-1,previous-2,previous-3,previous-4,previous-5
0,0,5.0,0,31.02125,31.02125,31.02125,31.02125,31.02125
1,1,5.0,0,31.02125,29.71069,29.71069,29.71069,29.71069
2,2,5.0,0,29.71069,31.02125,28.31853,28.31853,28.31853
3,3,5.0,0,28.31853,29.71069,31.02125,27.16955,27.16955
4,4,5.0,0,27.16955,28.31853,29.71069,31.02125,26.52516


In [89]:
# Compile model
model.compile(loss='mean_squared_error', optimizer='adam')

In [90]:
from tensorflow.keras.callbacks import EarlyStopping
# Split data into training and testing sets (add validation set if desired)
train_size = int(0.8 * len(X_reshaped))  # Adjust split ratio as needed
X_train, X_test = X_reshaped[:train_size], X_reshaped[train_size:]
y_train, y_test = Y_reshaped[:train_size], Y_reshaped[train_size:]


early_stopping = EarlyStopping(
    monitor='val_loss',  # Adjust based on your chosen metric
    patience=10,  # Adapt patience to your training requirements
    min_delta=0.001,  # Define the minimum improvement threshold
    restore_best_weights=True  # Choose whether to restore best weights
)


# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Evaluate performance on test set
predictions = model.predict(X_test)


Epoch 1/100
347/347 [==============================] - 33s 84ms/step - loss: 0.1964 - val_loss: 0.0608
Epoch 2/100
347/347 [==============================] - 3s 8ms/step - loss: 0.0296 - val_loss: 0.0341
Epoch 3/100
347/347 [==============================] - 2s 6ms/step - loss: 0.0246 - val_loss: 0.0303
Epoch 4/100
347/347 [==============================] - 2s 6ms/step - loss: 0.0232 - val_loss: 0.0289
Epoch 5/100
347/347 [==============================] - 2s 6ms/step - loss: 0.0219 - val_loss: 0.0277
Epoch 6/100
347/347 [==============================] - 2s 6ms/step - loss: 0.0214 - val_loss: 0.0262
Epoch 7/100
347/347 [==============================] - 2s 6ms/step - loss: 0.0204 - val_loss: 0.0265
Epoch 8/100
347/347 [==============================] - 3s 8ms/step - loss: 0.0200 - val_loss: 0.0272
Epoch 9/100
347/347 [==============================] - 2s 6ms/step - loss: 0.0197 - val_loss: 0.0301
Epoch 10/100
347/347 [==============================] - 2s 6ms/step - loss: 0.0191 - val_

In [91]:
print(y_test.shape)
y_test = y_test.reshape((2775, 1))
print(predictions.shape)

mse = mean_squared_error(y_test, predictions)
print('MSE:', mse)

(2775, 1, 1)
(2775, 1)
MSE: 0.012196355293261728


In [92]:
# Create the DataFrame
df_kiyas = pd.DataFrame({'y_test': y_test.flatten(), 'predictions': predictions.flatten()})

In [93]:
df_kiyas.head(10)

,y_test,predictions
0,-0.478595,-0.144190
1,-0.153077,-0.197970
2,0.014526,0.073543
3,-0.115122,-0.088408
4,-0.079133,-0.173139
5,-0.074965,-0.081279
6,-0.178841,-0.069337
7,-0.203974,-0.121587
8,-0.230975,-0.142387
9,-0.005938,-0.181952


In [94]:
y_test_original = scaler_y.inverse_transform(y_test)
predictions_original = scaler_y.inverse_transform(predictions)

In [95]:
import plotly.graph_objects as go

# Assuming 'df' is your DataFrame
fig = go.Figure()
columns = ['y_original', 'predict_original']
df_kiyas['y_original'] = y_test_original
df_kiyas['predict_original'] = predictions_original



In [96]:
# Add a trace for each column
for col in columns:
    fig.add_trace(go.Scatter(x=df_kiyas.index, y=df_kiyas[col], mode='lines', name=col))

fig.show()


In [97]:
df[train_size:].head(5)

,Date,Hour,Consumption,Weekday,Week_tag
11097,2023-04-08,9:00,33.96467,5.0,0
11098,2023-04-08,10:00,35.79573,5.0,0
11099,2023-04-08,11:00,36.73851,5.0,0
11100,2023-04-08,12:00,36.00923,5.0,0
11101,2023-04-08,13:00,36.21167,5.0,0


In [98]:
y_test_original

array([[33.96467],
       [35.79573],
       [36.73851],
       ...,
       [47.68305],
       [46.37095],
       [43.56002]])

DNN Trained ABove, now try on a new random data

##Validation test from a csv file

In [110]:
X.head(5)

,Hour,Weekday,Week_tag,previous-1,previous-2,previous-3,previous-4,previous-5
0,0,5.0,0,31.02125,31.02125,31.02125,31.02125,31.02125
1,1,5.0,0,31.02125,29.71069,29.71069,29.71069,29.71069
2,2,5.0,0,29.71069,31.02125,28.31853,28.31853,28.31853
3,3,5.0,0,28.31853,29.71069,31.02125,27.16955,27.16955
4,4,5.0,0,27.16955,28.31853,29.71069,31.02125,26.52516


In [133]:
df_test = pd.read_csv('/content/consum_test.csv')

def add_previous_consumption(df):
  for i in range(1, 6):
      df['previous-{}'.format(i)] = df['Consumption'].shift(i)
  for i in range(1, 6):
      df['previous-{}'.format(i)].fillna(df_test['Consumption'], inplace=True)

add_previous_consumption(df_test)


#Convert Hour
df_test['Hour'] = df_test['Hour'].apply(convert_hour)

df_test['Hour'] = df_test['Hour'].str[:2].astype('int')
#COnvert consumption
df_test['Consumption'] = df_test['Consumption'].apply(convert_consumption)

#convert kgup and previous data
columns_kgup = ['Consumption','previous-1','previous-2','previous-3','previous-4','previous-5']

for cols in columns_kgup:
  df_test[cols] = df_test[cols].apply(convert_consumption)




df_test.head(5)



,Date,Hour,Consumption,previous-1,previous-2,previous-3,previous-4,previous-5
0,02.08.2023,0,42.29243,42.29243,42.29243,42.29243,42.29243,42.29243
1,02.08.2023,1,40.54532,42.29243,40.54532,40.54532,40.54532,40.54532
2,02.08.2023,2,39.04091,40.54532,42.29243,39.04091,39.04091,39.04091
3,02.08.2023,3,37.87195,39.04091,40.54532,42.29243,37.87195,37.87195
4,02.08.2023,4,37.30196,37.87195,39.04091,40.54532,42.29243,37.30196


##Iterative appraoch

In [134]:
print(type(X_train))
df_test['Date'] = pd.to_datetime(df_test['Date'], dayfirst=True)
df_test['Weekday'] = 0

<class 'numpy.ndarray'>


In [135]:
 """add weekday to test data """
for i in range(len(df_test)):
  row = df_test.iloc[i]
  weekday = row['Date'].dayofweek
  df_test.loc[i, 'Weekday'] = weekday


In [136]:
df_test['Week_tag'] = 0
for i in range(len(df_test)):
  row = df_test.iloc[i]
  weekday = row['Weekday']
  if weekday in ['Sunday', 'Saturday']:
    df_test.loc[i, 'Week_tag'] = 1 #weekedn
  else:
    df_test.loc[i, 'Week_tag'] = 0 #weekin



In [137]:
df_test.head(5)

,Date,Hour,Consumption,previous-1,previous-2,previous-3,previous-4,previous-5,Weekday,Week_tag
0,2023-08-02,0,42.29243,42.29243,42.29243,42.29243,42.29243,42.29243,2,0
1,2023-08-02,1,40.54532,42.29243,40.54532,40.54532,40.54532,40.54532,2,0
2,2023-08-02,2,39.04091,40.54532,42.29243,39.04091,39.04091,39.04091,2,0
3,2023-08-02,3,37.87195,39.04091,40.54532,42.29243,37.87195,37.87195,2,0
4,2023-08-02,4,37.30196,37.87195,39.04091,40.54532,42.29243,37.30196,2,0


In [138]:
# Assuming your test data is in a DataFrame named 'test_data'
df_test = df_test[['Date','Hour','Consumption', 'Weekday', 'Week_tag', 'previous-1', 'previous-2', 'previous-3', 'previous-4', 'previous-5']]

df_test_X = df_test.drop(columns=['Date', 'Consumption'])
df_test_Y = df_test[['Consumption']]

#transform the test data
df_test_X = scaler_x.fit_transform(df_test_X)
# df_test_Y = scaler_y.fit_transform(df_test_Y)

#reshape them
# Reshape data for LSTM input
df_test_X = df_test_X.reshape((df_test_X.shape[0], 1, df_test_X.shape[1]))  # Assuming a single timestep
# df_test_Y = df_test_Y.reshape((df_test_Y.shape[0], 1, df_test_Y.shape[1]))  # Assuming a single timestep


In [139]:
pred_test = model.predict(df_test_X)

3/3 [==============================] - 0s 3ms/step


In [141]:
pred_test_real = scaler_y.inverse_transform(pred_test)

In [142]:
print(pred_test_real[:5])

[[40.590122]
 [42.01181 ]
 [37.57951 ]
 [38.68894 ]
 [38.90776 ]]


In [143]:
df_test_Y.head(5)

,Consumption
0,42.29243
1,40.54532
2,39.04091
3,37.87195
4,37.30196


In [147]:
df_test_Y['predictions'] = pred_test_real


<ipython-input-147-9bb77168a7e2>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [148]:
df_test_Y.head(2)

,Consumption,predictions
0,42.29243,40.590122
1,40.54532,42.011810


##Plot the real life test result with really bad data

In [150]:
# Add a trace for each column
fig2 = go.Figure()

columns_test =  ['Consumption', 'predictions']
for col in columns_test:
    fig2.add_trace(go.Scatter(x=df_test_Y.index, y=df_test_Y[col], mode='lines', name=col))

fig2.show()
